In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle
import warnings
warnings.filterwarnings('ignore')

In [2]:
## loading the dataset
df = pd.read_csv(r"C:\Users\himan\Desktop\Projects\Retention Risk Scoring Model\dataset\Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
## dropping unnecessary columns
df = df.drop(["RowNumber", "CustomerId", "Surname"], axis=1)

In [4]:
df.isnull().sum() ## checking for null values

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [5]:
## encoding the categorical columns
label_encoder_gender = LabelEncoder()
df["Gender"] = label_encoder_gender.fit_transform(df["Gender"])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [6]:
## Onehot encoding for 'Geography' column
from sklearn.preprocessing import OneHotEncoder
ohe_geo = OneHotEncoder()
geo_encoder = ohe_geo.fit_transform(df[["Geography"]])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [7]:
ohe_geo.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [8]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns = ohe_geo.get_feature_names_out(["Geography"]))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [9]:
## combining the encoded geography columns with the main dataframe
df = pd.concat([df.drop("Geography", axis=1), geo_encoded_df], axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
## saving the scalers and encoders
with open("label_encoder_gender.pkl", "wb") as file:
    pickle.dump(label_encoder_gender, file)

with open("onehot_encoder_geography.pkl", "wb") as file:
    pickle.dump(ohe_geo, file)

In [11]:
## dividing the dataset into features and target variable
X = df.drop("Exited", axis = 1)
y = df["Exited"]

In [12]:
## splitting the dataset into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [13]:
## Scaling the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
with open("scaler.pkl", "wb") as file: ## saving the file
    pickle.dump(scaler, file)

### Implementing ANN

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [16]:
## buildingf our ANN model
model = Sequential([
    Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    Dense(32, activation="relu"),
    Dense(1, activation="sigmoid")
])

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)

In [19]:
## compiling the model for backpropagation
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["accuracy"])

In [20]:
## set up the tensorboard 
log_directory = "log/fit/"
tensorboard_callback = TensorBoard(log_dir=log_directory, histogram_freq=1)

In [21]:
## early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

In [22]:
# training the model
history = model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs=100, callbacks=[tensorboard_callback, early_stopping])

Epoch 1/100
235/235 [==============================] - 1s 3ms/step - loss: 0.3963 - accuracy: 0.8348 - val_loss: 0.3565 - val_accuracy: 0.8588
Epoch 2/100
235/235 [==============================] - 0s 2ms/step - loss: 0.3576 - accuracy: 0.8545 - val_loss: 0.3485 - val_accuracy: 0.8636
Epoch 3/100
235/235 [==============================] - 0s 2ms/step - loss: 0.3501 - accuracy: 0.8583 - val_loss: 0.3492 - val_accuracy: 0.8588
Epoch 4/100
235/235 [==============================] - 0s 2ms/step - loss: 0.3421 - accuracy: 0.8600 - val_loss: 0.3489 - val_accuracy: 0.8604
Epoch 5/100
235/235 [==============================] - 0s 2ms/step - loss: 0.3400 - accuracy: 0.8601 - val_loss: 0.3497 - val_accuracy: 0.8632
Epoch 6/100
235/235 [==============================] - 0s 2ms/step - loss: 0.3371 - accuracy: 0.8639 - val_loss: 0.3442 - val_accuracy: 0.8588
Epoch 7/100
235/235 [==============================] - 0s 2ms/step - loss: 0.3349 - accuracy: 0.8628 - val_loss: 0.3457 - val_accuracy: 0.8628

In [23]:
model.save("retention_risk_scoring_model.keras")

In [24]:
## loading tensorboard 
%load_ext tensorboard

In [25]:
%tensorboard --logdir log/fit

Reusing TensorBoard on port 6007 (pid 48680), started 1:41:54 ago. (Use '!kill 48680' to kill it.)